# 노인 교통 사고 발생 지역 map

In [1]:
import requests, json

def get_location(address):
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
                      # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
                      # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
        headers = {"Authorization": "KakaoAK aa058a07922433baf688d53c0c06e5ea"}
        api_json = json.loads(str(requests.get(url,headers=headers).text))
        address = api_json['documents'][0]['address']
        rcd =str(address['y']) + ','+ str(address['x'])
        address_name = address['address_name']
        return rcd
        
    
    except:
        return 'r,c'

In [7]:
from google.colab import files
myfile = files.upload()

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
filename = '/content/drive/My Drive/miniproject/노인 교통사고/law.csv'

In [12]:
import pandas as pd
# 노인교통사고데이터와 법규위반 합해져있는 파일: law
silver_trans_csv = pd.read_csv(filename)
silver_trans_csv.head()

,지역구,지점명,도로명,중앙선침범,신호위반,안전거리미확보,불법유턴,과속,안전운전불이행,교차로위반,보행자보호위반,차로위반,직진우회전방해,기타,합계
0,강남구,수서역사거리,서울특별시 강남구 수서동 724 로즈데일오피스텔,0,2,5,0,0,6,1,0,1,0,1,16
1,강남구,양재전화국입구,서울특별시 강남구 도곡1동 207,0,4,2,0,0,11,1,0,0,0,0,18
2,강남구,신논현역 사거리,서울 강남구 강남대로 470,0,3,2,0,0,5,2,0,1,0,1,14
3,강남구,을지병원 앞 사거리,서울 강남구 논현동 279,0,3,2,0,0,7,4,0,1,0,0,17
4,강남구,대치우성아파트 사거리,서울 강남구 영동대로 230,0,7,2,0,0,3,0,0,0,0,0,12


In [13]:
# law파일 데이터에서 도로명만 뽑아서 위도 경도 변환
silver_trans_address = silver_trans_csv['도로명']
silver_trans_address

0      서울특별시 강남구 수서동 724 로즈데일오피스텔
1              서울특별시 강남구 도곡1동 207
2                 서울 강남구 강남대로 470
3                  서울 강남구 논현동 279
4                 서울 강남구 영동대로 230
                  ...            
737             서울 중랑구 상봉중앙로8길 41
738             서울 중랑구 망우동 535-10
739             서울 중랑구 면목동 1509-2
740                서울 중랑구 망우로 399
741             서울 중랑구 면목동 1509-2
Name: 도로명, Length: 742, dtype: object

In [14]:
# get_location 함수로 위도 경도 반환
silver_trans_latitude = []
silver_trans_longitude = []
for i in silver_trans_address:
    lat, lng = get_location(i).split(',')
    silver_trans_latitude.append(lat)
    silver_trans_longitude.append(lng)

In [52]:
# 교통 사고 발생 지역 위도 경도 데이터 프레임: trans_address_df 
trans_address_df = pd.DataFrame({ '지역구':silver_trans_csv["지역구"],
                                 "합계": silver_trans_csv["합계"],
                                 '도로명':silver_trans_csv["도로명"],
                           '위도':silver_trans_latitude,
                           '경도':silver_trans_longitude})
exc = trans_address_df[trans_address_df["위도"].str.contains("r")].index
# 교통사고 발생지역 데이터프레임에서 예외 처리된 값 제거
trans_exc = trans_address_df.drop(exc)
trans_exc.to_csv("trans_exc.csv")
trans_exc

,지역구,합계,도로명,위도,경도
0,강남구,16,서울특별시 강남구 수서동 724 로즈데일오피스텔,37.4873267937748,127.103140081887
1,강남구,18,서울특별시 강남구 도곡1동 207,37.4857245085271,127.042517787926
2,강남구,14,서울 강남구 강남대로 470,37.5043293460531,127.025061202164
3,강남구,17,서울 강남구 논현동 279,37.519433345399,127.027816287344
4,강남구,12,서울 강남구 영동대로 230,37.5009601493207,127.068471209933
...,...,...,...,...,...
737,중랑구,5,서울 중랑구 상봉중앙로8길 41,37.6025121881772,127.089346030689
738,중랑구,9,서울 중랑구 망우동 535-10,37.5994933398436,127.095767092241
739,중랑구,8,서울 중랑구 면목동 1509-2,37.5784209373757,127.081742369843
740,중랑구,5,서울 중랑구 망우로 399,37.5993004342889,127.096545481223


In [53]:
# 교통사고 발생지역 데이터프레임 인덱스 재정의
trans_group = trans_address_df.groupby("지역구")["합계"].sum()
trans_group

지역구
강남구     327
강동구     207
강북구     297
강서구     216
관악구     288
광진구     180
구로구     238
금천구     203
노원구     266
도봉구     168
동대문구    327
동작구     313
마포구     201
서대문구    225
서초구     292
성동구     170
성북구     286
송파구     310
양천구     239
영등포구    285
용산구     184
은평구     225
종로구     274
중구      254
중랑구     279
Name: 합계, dtype: int64

In [39]:
import requests
import json

# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

In [ ]:
# pip install ipyleaflet # 설치하고
# marker cluster로 지도에 마커 찍기 
#사용 방식
from ipyleaflet import Map, Marker, MarkerCluster
marker = []
m = Map(center=(37.5, 127), zoom=5)
for i in range(len(silver_trans_latitude)):
    marker.append(Marker(location = (silver_trans_latitude[i], silver_trans_longitude[i])))

marker_cluster = MarkerCluster(markers = marker)

m.add_layer(marker_cluster)

m

Map(center=[37.5, 127], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [40]:
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

In [55]:
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

m.choropleth(
    geo_data=state_geo,
    legend_name ='사건 발생 건수',
    data= trans_group,
    columns=['지역구', '합계'],
    key_on='feature.properties.name',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.3,
    color = 'gray'
)

for i in range(len(trans_exc)):
  latitude = trans_exc.iloc[i]["위도"]
  longitude = trans_exc.iloc[i]["경도"]
  location = (latitude, longitude)

  folium.CircleMarker(location, radius = 10, color = color, fill_color = color, fill_opacity = 0.1, opacity = 0.0).add_to(m)

folium.LayerControl(collapsed = False).add_to(m)
m

/usr/local/lib/python3.8/dist-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [17]:
x = []
y = []
for i in range(len(trans_exc['위도'])):
    if trans_exc['위도'][i] == 0.0 or trans_exc['경도'][i] == 0.0:
        pass
    else:
        x.append(trans_exc['위도'][i])
        y.append(trans_exc['경도'][i])
print('x갯수: ',len(x))
print('y갯수: ',len(y))

x갯수:  740
y갯수:  740


In [20]:
# foliumn으로 교통사고발생지역 지도에 마커표시 
import folium
map_osm = folium.Map(location=[x[13],y[13]],zoom_start=10)
for i in range(len(x)):
    folium.Marker([x[i],y[i]], icon=folium.Icon(color='red', icon='info-sign')).add_to(map_osm)
# 지도에서 범위 지정
folium.CircleMarker(location=[x[13],y[13]],popup='서울', radius=300, color="#3186CC", fill_color="#3186CC").add_to(map_osm)
#map저장
map_osm.save('Seoul_trans.html')

In [21]:
map_osm

In [ ]:
ㅁ